# Inference the trained mode

In this notebook you can try out a trained model.

In [22]:
import os
import numpy as np
import tensorflow as tf
import pyaudio
import keyboard
import time
import wave

In [23]:
def load_model(model_name):
    model_path = os.path.join("models",model_name)
    return tf.saved_model.load(model_path)

In [24]:
model = load_model("spec_classifier_01")

In [30]:
print("Deleting tmp content")
tmp_dir = "data/tmp"
for path in os.listdir(tmp_dir):
    if path == ".ipynb_checkpoints":
        continue
    full_path = os.path.join(tmp_dir,path)
    os.remove(full_path)

chunk = 1024  # Record in chunks of 1024 samples
sample_format = pyaudio.paInt16  # 16 bits per sample
channels = 2
fs = 16000  # Record at 16000 samples per second

p = pyaudio.PyAudio()  # Create an interface to PortAudio


print("Press 's' key to start recording")
print("Press 'q' key to quit recording")
while(True):
    if keyboard.is_pressed("s"):
        break
    
stream = p.open(format=sample_format,
                channels=channels,
                rate=fs,
                frames_per_buffer=chunk,
                input=True)

frames = []  # Initialize array to store frames
import threading as th

keep_going = True
def key_capture_thread():
    global keep_going
    while(True):
        if keyboard.is_pressed("q"):        
            keep_going = False
            break
        time.sleep(0.01)

th.Thread(target=key_capture_thread, args=(), name='key_capture_thread', daemon=True).start()

print("Starting audio recording.")
while keep_going:
    # Store data in chunks for 3 seconds
    for i in range(0, int(fs / chunk)):
        data = stream.read(chunk)
        frames.append(data)

# Stop and close the stream 
stream.stop_stream()
stream.close()
# Terminate the PortAudio interface
p.terminate()

print('Finished recording. Saving audio.')
filename=os.path.join(tmp_dir,"inference.wav")


# Save the recorded data as a WAV file
wf = wave.open(filename, 'wb')
wf.setnchannels(channels)
wf.setsampwidth(p.get_sample_size(sample_format))
wf.setframerate(fs)
wf.writeframes(b''.join(frames))
wf.close()

print("Inferencing:")
result = model(filename)
result = str(result['class_names'].numpy())
print(result)

Deleting tmp content
Press 's' key to start recording
Press 'q' key to quit recording
Starting audio recording.
Finished recording. Saving audio.
Inferencing:
[b'go']
